In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import cohen_kappa_score as kappa_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from skopt import gp_minimize
from skopt.space import Real, Integer
from sklearn.preprocessing import StandardScaler
kappa_scorer = make_scorer(kappa_score)
from keras.constraints import maxnorm
import os
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler

import glob
import cv2
import os
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import concatenate
import numpy as np
import argparse
import locale
from keras.utils import np_utils
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import RMSprop
import pandas as pd
import tensorflow as tf
from keras import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.layers import Dropout, Dense, np
from keras.optimizers import SGD
from keras.utils import to_categorical
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

In [2]:
train_df = pd.read_csv("../input/all/train.csv")
test_df = pd.read_csv("../input/all/test/test.csv")

In [3]:
train_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [4]:
cat_cols = ['Type','Age','Breed1','Breed2', 'Gender', 'Color1', 'Color2', 'Color3', 'MaturitySize', 
          'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized','Health', 'Quantity','State','VideoAmt','PhotoAmt']

In [5]:
num_cols = ['Fee']

In [6]:
text_cols = ['Description']

## Handling categorical columns

In [7]:
embed_sizes = [len(train_df[col].unique()) + 1 for col in cat_cols]

In [8]:
print(embed_sizes)

[3, 107, 177, 136, 4, 8, 8, 7, 5, 4, 4, 4, 4, 4, 20, 15, 10, 32]


## Handling numerical columns

In [9]:
print('scaling num_cols')
for col in num_cols:
    print('scaling {}'.format(col))
    col_mean = train_df[col].mean()
    train_df[col].fillna(col_mean, inplace=True)
    test_df[col].fillna(col_mean, inplace=True)
    scaler = StandardScaler()
    train_df[col] = scaler.fit_transform(train_df[col].values.reshape(-1, 1))
    test_df[col] = scaler.transform(test_df[col].values.reshape(-1, 1))

scaling num_cols
scaling Fee


/Users/zekunzhao/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/zekunzhao/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/zekunzhao/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Handling text columns

In [10]:
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [11]:
print('getting embeddings')
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in tqdm(open('//Users/zekunzhao/Downloads/wiki-news-300d-1M-subword.vec')))


1175it [00:00, 11746.01it/s]

getting embeddings


1000000it [01:22, 12146.32it/s]


In [12]:
num_words = 20000
maxlen = 80
embed_size = 300

In [13]:
train_df['Description'] = train_df['Description'].astype(str).fillna('no text')
test_df['Description'] = test_df['Description'].astype(str).fillna('no text')

In [14]:
print("   Fitting tokenizer...")
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_df['Description'].values.tolist())

   Fitting tokenizer...


In [15]:
train_df['Description'] = tokenizer.texts_to_sequences(train_df['Description'])
test_df['Description'] = tokenizer.texts_to_sequences(test_df['Description'])

In [16]:
word_index = tokenizer.word_index
nb_words = min(num_words, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= num_words: continue
    try:
        embedding_vector = embeddings_index[word]
    except KeyError:
        embedding_vector = None
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector


In [17]:
def get_input_features(df):
    X = {'description':pad_sequences(df['Description'], maxlen=maxlen)}
    X['numerical'] = np.array(df[num_cols])
    for cat in cat_cols:
        X[cat] = np.array(df[cat])
    return X

## Define NN Model

In [20]:
from keras.layers import Input, Embedding, Concatenate, Flatten, Dense, Dropout, BatchNormalization,LSTM, CuDNNLSTM, SpatialDropout1D
from keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPool1D
from keras.models import Model
from keras.optimizers import  Adam

def prepare_data(data):
    pet_id = data.PetID

    # Remove unused features
    data.drop(['RescuerID', 'Description', 'PetID', 'State'], axis=1, inplace=True)

    # Apply binning to ages
    data['Age'] = pd.cut(data['Age'], [-1, 2, 3, 6, 255], labels=[0, 1, 2, 3])

    # Apply binning to fee
    data['Fee'] = pd.cut(data['Fee'], [-1, 50, 100, 200, 3000], labels=[0, 1, 2, 3])

    # Apply binning to photo amount
    data['PhotoAmt'] = pd.cut(data['PhotoAmt'], [-1, 1, 5, 10, 100], labels=[0, 1, 2, 3])

    # Apply binning to video amount
    data['VideoAmt'] = pd.cut(data['VideoAmt'], [-1, 1, 100], labels=[0, 1])

    # Replace names with 1 is present, 0 if not present
    data.loc[data['Name'].notnull(), 'Name'] = 1
    data.loc[data['Name'].isnull(), 'Name'] = 0

    # Fill missing continuous data
    data_continuous = data.select_dtypes(exclude=['object'])
    data_continuous.fillna(0, inplace=True)

    # Fill missing string data
    data_categorical = data.select_dtypes(include=['object'])
    data_categorical.fillna('NONE', inplace=True)

    final_data = data_continuous.merge(data_categorical, left_index=True, right_index=True)

    return final_data, data_categorical, data_continuous, pet_id, data.shape[1]


categorical_inputs = []
for cat in cat_cols:
    categorical_inputs.append(Input(shape=[1], name=cat))

categorical_embeddings = []
for i, cat in enumerate(cat_cols):
    categorical_embeddings.append(
        Embedding(embed_sizes[i], 10)(categorical_inputs[i]))

categorical_logits = Concatenate()([Flatten()(cat_emb) for cat_emb in categorical_embeddings])
categorical_logits = Dense(256, activation = 'relu')(categorical_logits)


numerical_inputs = Input(shape=[len(num_cols)], name='numerical')
numerical_logits = numerical_inputs
numerical_logits = BatchNormalization()(numerical_logits)
numerical_logits = Dense(128, activation = 'relu')(numerical_logits)

text_inp = Input(shape=[maxlen], name='description')
text_embed = Embedding(nb_words, embed_size, weights=[embedding_matrix], trainable=False)(text_inp)
text_logits = SpatialDropout1D(0.2)(text_embed)
text_logits = Bidirectional(LSTM(64, return_sequences=True))(text_logits)
avg_pool = GlobalAveragePooling1D()(text_logits)
max_pool = GlobalMaxPool1D()(text_logits)
text_logits = Concatenate()([avg_pool, max_pool])

x = Concatenate()([categorical_logits, text_logits, numerical_logits])
x = BatchNormalization()(x)
x = Dense(128, activation = 'relu')(x)
x = Dropout(0.3)(x)
out = Dense(1, activation = 'sigmoid')(x)

model = Model(inputs=[text_inp] + categorical_inputs + [numerical_inputs],outputs=out)
model.compile(optimizer=Adam(lr = 0.0001), loss = 'mse')

In [21]:
from sklearn.model_selection import train_test_split

#for i, l in enumerate(tr_df['AdoptionSpeed'].values):
#    y_train[i,l] = 1
#for i, l in enumerate(val_df['AdoptionSpeed'].values):
#    y_valid[i,l] = 1
tr_df, val_df = train_test_split(train_df, test_size = 0.6, random_state = 23)

In [22]:
# from keras.utils.np_utils import to_categorical

In [23]:
tr_df['AdoptionSpeed'].values.shape

(5997,)

In [24]:
y_train = tr_df['AdoptionSpeed'].values / 4
y_valid = val_df['AdoptionSpeed'].values / 4

In [25]:
y_train = np_utils.to_categorical(tr_df['AdoptionSpeed'], num_classes=5)
y_valid = np_utils.to_categorical(val_df['AdoptionSpeed'], num_classes=5)
# y_test = np_utils.to_categorical(test_df['AdoptionSpeed'], num_classes=5)

In [26]:
# X_train = get_input_features(tr_df)
# X_valid = get_input_features(val_df)
# X_test = get_input_features(test_df)
test_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt
0,1,Puppy,2,307,0,1,1,0,0,2,...,2,1,1,1.641842,41326,4475f31553f0170229455e3c5645644f,0,"[72, 5, 590, 6, 3, 269, 32, 38, 13, 3234, 16, ...",378fcc4fc,3.0
1,2,London,24,266,0,1,2,7,0,2,...,1,1,1,-0.271132,41326,4475f31553f0170229455e3c5645644f,0,"[427, 736, 39, 20, 54, 6, 619, 667]",73c10e136,1.0
2,2,Snowball,20,266,0,2,7,0,0,2,...,1,1,1,1.641842,41326,4475f31553f0170229455e3c5645644f,0,"[7860, 357, 219, 47, 43, 9, 5, 74, 3, 369, 108...",72000c4c5,1.0
3,2,Malibu,5,266,252,2,1,6,7,2,...,1,1,1,1.004184,41326,4475f31553f0170229455e3c5645644f,0,"[13607, 101, 753, 259, 105, 124, 88, 165, 1, 1...",e147a4b9f,1.0
4,1,Lala Girl,6,307,0,2,1,2,7,2,...,1,1,1,1.641842,41326,4475f31553f0170229455e3c5645644f,0,"[3696, 894, 21, 191, 148, 3, 141, 158, 52, 179...",43fbba852,1.0


In [36]:
continuous = ['Type','Age','Breed1','Breed2', 'Gender', 'Color1', 'Color2', 'Color3', 'MaturitySize', 
          'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized','Health', 'Quantity','State','VideoAmt','PhotoAmt']
# zipcodes = tr_df[uncontinuous].value_counts().keys().tolist()
# counts = tr_df[uncontinuous].value_counts().tolist()

# 	# loop over each of the unique zip codes and their corresponding
# 	# count
# for (zipcode, count) in zip(zipcodes, counts):
# 		# the zip code counts for our housing dataset is *extremely*
# 		# unbalanced (some only having 1 or 2 houses per zip code)
# 		# so let's sanitize our data by removing any houses with less
# 		# than 25 houses per zip code
# 	if count < 25:
# 		idxs = tr_df[tr_df[uncontinuous] == zipcode].index
# 		tr_df.drop(idxs, inplace=True)
        
# zipBinarizer = LabelBinarizer().fit(tr_df[uncontinuous])
# trainCategorical = zipBinarizer.transform(tr_df[uncontinuous])

cs = MinMaxScaler()
trainContinuous = cs.fit_transform(tr_df[continuous])
trainContinuous2 = cs.fit_transform(val_df[continuous])
trainContinuous3 = cs.fit_transform(test_df[continuous])

/Users/zekunzhao/venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/zekunzhao/venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/zekunzhao/venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [39]:
x_train = np.hstack([trainContinuous])
x_valid = np.hstack([trainContinuous2])
x_test = np.hstack([trainContinuous3])
model = Sequential([
    Dense(64, input_dim=18),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(16),
    Activation('relu'),
    Dense(5),
    Activation('sigmoid'),
])
rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# We add metrics to get more results you want to see
#categorical_crossentropy--mse
model.compile(optimizer=rmsprop,
              loss='mse',
              metrics=['accuracy'])
model.fit(x_train, y_train,validation_data = (x_valid,y_valid),epochs=260, batch_size=10)

Train on 5997 samples, validate on 8996 samples
Epoch 1/260
5997/5997 [==============================] - 2s 268us/step - loss: 0.1569 - acc: 0.2738 - val_loss: 0.1520 - val_acc: 0.2807
Epoch 2/260
5997/5997 [==============================] - 1s 186us/step - loss: 0.1497 - acc: 0.3167 - val_loss: 0.1493 - val_acc: 0.3226
Epoch 3/260
5997/5997 [==============================] - 1s 184us/step - loss: 0.1484 - acc: 0.3308 - val_loss: 0.1477 - val_acc: 0.3420
Epoch 4/260
5997/5997 [==============================] - 1s 182us/step - loss: 0.1472 - acc: 0.3408 - val_loss: 0.1474 - val_acc: 0.3430
Epoch 5/260
5997/5997 [==============================] - 1s 184us/step - loss: 0.1464 - acc: 0.3535 - val_loss: 0.1470 - val_acc: 0.3505
Epoch 6/260
5997/5997 [==============================] - 1s 177us/step - loss: 0.1455 - acc: 0.3632 - val_loss: 0.1464 - val_acc: 0.3606
Epoch 7/260
5997/5997 [==============================] - 1s 177us/step - loss: 0.1448 - acc: 0.3642 - val_loss: 0.1471 - val_acc: 

5997/5997 [==============================] - 1s 180us/step - loss: 0.1290 - acc: 0.4814 - val_loss: 0.1577 - val_acc: 0.3536
Epoch 61/260
5997/5997 [==============================] - 1s 179us/step - loss: 0.1291 - acc: 0.4812 - val_loss: 0.1556 - val_acc: 0.3470
Epoch 62/260
5997/5997 [==============================] - 1s 177us/step - loss: 0.1290 - acc: 0.4766 - val_loss: 0.1575 - val_acc: 0.3474
Epoch 63/260
5997/5997 [==============================] - 1s 177us/step - loss: 0.1289 - acc: 0.4842 - val_loss: 0.1571 - val_acc: 0.3448
Epoch 64/260
5997/5997 [==============================] - 1s 179us/step - loss: 0.1284 - acc: 0.4791 - val_loss: 0.1555 - val_acc: 0.3552
Epoch 65/260
5997/5997 [==============================] - 1s 178us/step - loss: 0.1282 - acc: 0.4816 - val_loss: 0.1582 - val_acc: 0.3402
Epoch 66/260
5997/5997 [==============================] - 1s 178us/step - loss: 0.1280 - acc: 0.4826 - val_loss: 0.1602 - val_acc: 0.3445
Epoch 67/260
5997/5997 [=======================

5997/5997 [==============================] - 1s 202us/step - loss: 0.1195 - acc: 0.5426 - val_loss: 0.1680 - val_acc: 0.3465
Epoch 120/260
5997/5997 [==============================] - 1s 204us/step - loss: 0.1190 - acc: 0.5438 - val_loss: 0.1696 - val_acc: 0.3415
Epoch 121/260
5997/5997 [==============================] - 1s 191us/step - loss: 0.1193 - acc: 0.5396 - val_loss: 0.1668 - val_acc: 0.3483
Epoch 122/260
5997/5997 [==============================] - 1s 189us/step - loss: 0.1188 - acc: 0.5426 - val_loss: 0.1671 - val_acc: 0.3486
Epoch 123/260
5997/5997 [==============================] - 1s 180us/step - loss: 0.1188 - acc: 0.5423 - val_loss: 0.1657 - val_acc: 0.3455
Epoch 124/260
5997/5997 [==============================] - 1s 179us/step - loss: 0.1184 - acc: 0.5466 - val_loss: 0.1685 - val_acc: 0.3514
Epoch 125/260
5997/5997 [==============================] - 1s 201us/step - loss: 0.1181 - acc: 0.5399 - val_loss: 0.1705 - val_acc: 0.3306
Epoch 126/260
5997/5997 [================

5997/5997 [==============================] - 1s 172us/step - loss: 0.1148 - acc: 0.5686 - val_loss: 0.1840 - val_acc: 0.3266
Epoch 179/260
5997/5997 [==============================] - 1s 181us/step - loss: 0.1142 - acc: 0.5629 - val_loss: 0.1755 - val_acc: 0.3351
Epoch 180/260
5997/5997 [==============================] - 1s 190us/step - loss: 0.1132 - acc: 0.5708 - val_loss: 0.1806 - val_acc: 0.3409
Epoch 181/260
5997/5997 [==============================] - 1s 192us/step - loss: 0.1133 - acc: 0.5738 - val_loss: 0.1785 - val_acc: 0.3434
Epoch 182/260
5997/5997 [==============================] - 1s 197us/step - loss: 0.1129 - acc: 0.5705 - val_loss: 0.1801 - val_acc: 0.3430
Epoch 183/260
5997/5997 [==============================] - 1s 200us/step - loss: 0.1137 - acc: 0.5691 - val_loss: 0.1758 - val_acc: 0.3540
Epoch 184/260
5997/5997 [==============================] - 1s 204us/step - loss: 0.1142 - acc: 0.5634 - val_loss: 0.1799 - val_acc: 0.3457
Epoch 185/260
5997/5997 [================

5997/5997 [==============================] - 1s 210us/step - loss: 0.1106 - acc: 0.5816 - val_loss: 0.1844 - val_acc: 0.3507
Epoch 238/260
5997/5997 [==============================] - 1s 193us/step - loss: 0.1112 - acc: 0.5786 - val_loss: 0.1777 - val_acc: 0.3428
Epoch 239/260
5997/5997 [==============================] - 1s 185us/step - loss: 0.1123 - acc: 0.5805 - val_loss: 0.1752 - val_acc: 0.3566
Epoch 240/260
5997/5997 [==============================] - 1s 189us/step - loss: 0.1120 - acc: 0.5743 - val_loss: 0.1771 - val_acc: 0.3470
Epoch 241/260
5997/5997 [==============================] - 1s 189us/step - loss: 0.1111 - acc: 0.5883 - val_loss: 0.1815 - val_acc: 0.3415
Epoch 242/260
5997/5997 [==============================] - 1s 194us/step - loss: 0.1104 - acc: 0.5835 - val_loss: 0.1733 - val_acc: 0.3427
Epoch 243/260
5997/5997 [==============================] - 1s 199us/step - loss: 0.1115 - acc: 0.5820 - val_loss: 0.1795 - val_acc: 0.3487
Epoch 244/260
5997/5997 [================

In [ ]:
# print('\nTesting ------------')
# # Evaluate the model with the metrics we defined earlier
# loss, accuracy = model.evaluate(x_test, y_test)

# print('test loss: ', loss)
# print('test accuracy: ', accuracy)


In [40]:
print(x_train)

[[0.         0.00784314 1.         ... 0.84615385 0.         0.06666667]
 [1.         0.         0.86644951 ... 0.13186813 0.         0.06666667]
 [1.         0.02352941 0.86319218 ... 0.84615385 0.         0.16666667]
 ...
 [1.         0.02352941 0.86644951 ... 0.84615385 0.         0.13333333]
 [1.         0.00784314 0.96742671 ... 0.02197802 0.         0.16666667]
 [1.         0.00784314 0.85993485 ... 0.84615385 0.125      0.16666667]]


In [ ]:
#y_pred2 = np.argmax(y_pred,axis = 1)
#y_pred2.shape